# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [5]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
browser = Browser('chrome')

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 117.0.5938.132 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00AAA813+48355]
	(No symbol) [0x00A3C4B1]
	(No symbol) [0x00945358]
	(No symbol) [0x009661AC]
	(No symbol) [0x00961EF3]
	(No symbol) [0x00960579]
	(No symbol) [0x00990C55]
	(No symbol) [0x0099093C]
	(No symbol) [0x0098A536]
	(No symbol) [0x009682DC]
	(No symbol) [0x009693DD]
	GetHandleVerifier [0x00D0AABD+2539405]
	GetHandleVerifier [0x00D4A78F+2800735]
	GetHandleVerifier [0x00D4456C+2775612]
	GetHandleVerifier [0x00B351E0+616112]
	(No symbol) [0x00A45F8C]
	(No symbol) [0x00A42328]
	(No symbol) [0x00A4240B]
	(No symbol) [0x00A34FF7]
	BaseThreadInitThunk [0x762C7BA9+25]
	RtlInitializeExceptionChain [0x771BB79B+107]
	RtlClearBits [0x771BB71F+191]


### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
mars_html = browser.html
mars_soup = soup(mars_html,'html.parser')


In [ ]:
# Extract all rows of data
table = mars_soup.find('table', class_='table')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
columns = []
data = []

# Loop through the rows

headings = table.find_all('th')
for heading in headings:
    columns.append(heading.get_text())

# Loop through the scraped data to create a list of rows

rows = table.find_all('tr', class_='data-row')

for row in rows:
    items = []
    row_data = row.find_all('td')
    for result in row_data:
        items.append(result.text)
    data.append(items)

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_weather_df = pd.DataFrame(data, columns = columns)


In [ ]:
# Confirm DataFrame was created successfully
mars_weather_df



### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_weather_df.dtypes


In [ ]:
# Change data types for data analysis
mars_weather_df[mars_weather_df.columns[1]] = pd.to_datetime(mars_weather_df[mars_weather_df.columns[1]])
mars_weather_df[mars_weather_df.columns[2]] = mars_weather_df[mars_weather_df.columns[2]].astype(int) 
mars_weather_df[mars_weather_df.columns[3]] = mars_weather_df[mars_weather_df.columns[3]].astype(int)
mars_weather_df[mars_weather_df.columns[4]] = mars_weather_df[mars_weather_df.columns[4]].astype(int) 
mars_weather_df[mars_weather_df.columns[5]] = mars_weather_df[mars_weather_df.columns[5]].astype(float) 
mars_weather_df[mars_weather_df.columns[6]] = mars_weather_df[mars_weather_df.columns[6]].astype(float) 

In [ ]:
# Confirm type changes were successful by examining data types again
mars_weather_df.dtypes


### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
print (f"The number of months on mars is: {mars_weather_df['month'].max()}") 


In [ ]:
# 2. How many Martian days' worth of data are there?
print (f"The number of Martian days (sols) for which data was collected: {mars_weather_df['sol'].max()}")


In [ ]:
# 3. What is the average low temperature by month?
mars_month_temp = mars_weather_df[['month','min_temp']].groupby(by='month').mean()

mars_month_temp['min_temp'] = mars_month_temp['min_temp'].round(decimals = 2)

mars_month_temp

In [ ]:
# Plot the average temperature by month
fig, ax = plt.subplots(figsize=[6,5])
ax = plt.bar(mars_month_temp.index,mars_month_temp.min_temp)
plt.xticks(ticks = range(1,13,1))
plt.xlabel('Month')
plt.ylabel('Temperature in Celsius')
plt.title('Average Temperature by Martian Month')
plt.legend(['Average Min Temp'],loc='best')
plt.show()


In [ ]:
# Identify the coldest and hottest months in Curiosity's location
print (f"The coldest month on Mars is: {mars_month_temp['min_temp'].idxmin()}") 

print (f"The hottest month on Mars is: {mars_month_temp['min_temp'].idxmax()}")


In [ ]:
# 4. Average pressure by Martian month
mars_month_pressure = mars_weather_df[['month','pressure']].groupby(by='month').mean()

mars_month_pressure['pressure'] = mars_month_pressure['pressure'].round(decimals = 2)

mars_month_pressure



In [ ]:
# Plot the average pressure by month
fig, ax = plt.subplots(figsize=[8,5])
ax = plt.bar(mars_month_pressure.index,mars_month_pressure.pressure)
plt.xticks(ticks = range(1,13,1))
plt.xlabel('Month')
plt.ylabel('Average Pressure')
plt.title('Average Pressure by Martian Month')
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
fig, ax = plt.subplots(figsize=[15,7])
ax = plt.plot(mars_weather_df.terrestrial_date,mars_weather_df.min_temp)
plt.grid(visible=True, which='both')
plt.xlabel('Number of terrestrial days')
plt.ylabel('Minimum Temperature')
plt.title('Min Temp on Mars per Earth Day')
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_weather_df.to_csv("./Outputs/mars_weather.csv")

mars_month_temp.to_csv("./Outputs/mars_month_temp.csv")

mars_month_pressure.to_csv("./Outputs/mars_month_pressure.csv")


In [ ]:
browser.quit()